In [1]:
import pyspark
SPARK_VERSION = '3.1.2'
SCALA_VERSION = '2.12'

### adiciona package adicional
import findspark
findspark.add_packages(['org.apache.spark:spark-sql-kafka-0-10_' + SCALA_VERSION + ':' + SPARK_VERSION ])
findspark.init()

import pandas as pd
import pyarrow

import mysql.connector as mysql 
con = mysql.connect(host='localhost',database='teste',user='saulo',password='Olivetti@5!', auth_plugin='mysql_native_password')
cursor = con.cursor()

### extrai coluna valor do dictionary e grava arquivo .csv  
def ProcessaLinha(df_grava):
    df_montado = pd.DataFrame([df_grava])
    valor = str(df_grava['value'])
    timestamp = str(df_grava['timestamp'])
    df_montado.to_csv('/tmp/people.csv', mode='a', index=False, sep='|', header=False)
    comando = f'''insert into kafka values ("{valor}", "{timestamp}")'''
    print(comando)


In [ ]:
### inicializa instancia spark 
spark = pyspark.sql.SparkSession \
          .builder \
          .appName("APP") \
          .getOrCreate()

### inicia leitura streaming do inicio 
df = spark\
      .readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "localhost:9092") \
      .option("subscribe", "saulo-teste") \
      .option("startingOffsets",""" {"saulo-teste":{"0":0}}""") \
      .option("failOnDataLoss", "false") \
      .load()


###query = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "topic","CAST(timestamp AS STRING)", "CAST(offset AS STRING)", "CAST(partition AS STRING)" ) \
query = df.writeStream \
    .foreach(ProcessaLinha) \
    .start()

query.awaitTermination()

:: loading settings :: url = jar:file:/home/saulos_pires/.local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/saulos_pires/.ivy2/cache
The jars for the packages stored in: /home/saulos_pires/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-27d2c6fb-ae0b-4cd1-9432-d2e525318435;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 435ms :: artifacts dl 16ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 

insert into kafka values ("bytearray(b'complemento de mensagem')", "2022-05-30 18:41:20.697000")
